In [42]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras_vggface.vggface import VGGFace

# using MTCNN for face detection and alignment
from mtcnn import MTCNN

# using dlib for face detection and alignment
from eye_alignment_multiple import align_faces	

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import re
from sklearn.model_selection import train_test_split

### Use MTCNN for face detection and alignment

In [7]:
# run this if use MTCNN for face detection and alignment
face_detector = MTCNN()
print("Face detector model loded...")

Face detector model loded...


In [ ]:
# Lists to store images and labels
dataset_images = []
dataset_labels = []
label_map = {}  # Mapping of labels to indices
label_index = 0

def MTCNN_add_image(image_path):
    """Loads images and extracts embeddings for training."""
    global label_index

    root, _ = os.path.splitext(image_path)
    label = os.path.split(root)[-1]
    match = re.search(r"^(\w+)_", label)
    label = match.group(1)

    if label not in label_map:
        label_map[label] = label_index
        label_index += 1

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = face_detector.detect_faces(image) # use MTCNN for face detection and alignment

    if not faces:
        print(f"No face found in `{label}`, skipping.")
        return

    if len(faces) > 1:
        print(f"Multiple faces found in `{label}`, skipping.")
        return
    x, y, w, h = faces[0]["box"]
    x1, y1 = max(0, x), max(0, y)
    x2, y2 = min(image.shape[1], x + w), min(image.shape[0], y + h)

    cropped_face = image[y1:y2, x1:x2]
    resized_face = cv2.resize(cropped_face, (224, 224))
    resized_face = resized_face.reshape(1, 224, 224, 3)

    dataset_images.append(resized_face)
    dataset_labels.append(label_map[label])

# Load images from dataset
for dir, _, files in os.walk("../Member Photos"):
    for file in files:
        MTCNN_add_image(os.path.join(dir, file))

dataset_images = np.array(dataset_images)
dataset_labels = np.array(dataset_labels)
dataset_labels = to_categorical(dataset_labels, num_classes=len(label_map))

print(f"Loaded {len(dataset_images)} images for training.")

### Use dlib for face detection and alignment

In [26]:
# Lists to store images and labels
dataset_images = []
dataset_labels = []
label_map = {}  # Mapping of labels to indices
label_index = 0

def dlib_add_image(image_path):
    """Loads images and extracts embeddings for training."""
    global label_index

    root, _ = os.path.splitext(image_path)
    label = os.path.split(root)[-1]
    match = re.search(r"^(\w+)_", label)
    label = match.group(1)

    if label not in label_map:
        label_map[label] = label_index
        label_index += 1

    image = cv2.imread(image_path)
    faces, bounding_boxes = align_faces(image)  # use dlib for face detection and alignment

    if not faces:
        print(f"No face found in `{label}`, skipping.")
        return

    if len(faces) > 1:
        print(f"Multiple faces found in `{label}`, skipping.")
        return

    resized_face = cv2.resize(faces[0], (224, 224))

    dataset_images.append(resized_face)
    dataset_labels.append(label_map[label])

# Load images from dataset
for dir, _, files in os.walk("../Member Photos"):
    for file in files:
        dlib_add_image(os.path.join(dir, file))

dataset_images = np.array(dataset_images)
dataset_labels = np.array(dataset_labels)
dataset_labels = to_categorical(dataset_labels, num_classes=len(label_map))

print(f"Loaded {len(dataset_images)} images for training.")

No face detected!
No face found in `aaryan`, skipping.
No face detected!
No face found in `aaryan`, skipping.
No face detected!
No face found in `ethan`, skipping.
No face detected!
No face found in `ethan`, skipping.
No face detected!
No face found in `ethan`, skipping.
No face detected!
No face found in `ethan`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
No face detected!
No face found in `jinwei`, skipping.
Loaded 78 images for training.


In [36]:
print("shape of dataset_images:", dataset_images.shape)
print("shape of dataset_labels:", dataset_labels.shape)

shape of dataset_images: (78, 224, 224, 3)
shape of dataset_labels: (78, 6)


### Load ResNet50 for feature extraction

In [ ]:
BASE = r"C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/"
checkpoint_path = BASE + "checkpoints/resnet50_face_recognition.h5"

train_images, val_images, train_labels, val_labels = train_test_split(
    dataset_images, dataset_labels, test_size=0.2, stratify=dataset_labels, random_state=42
)

# Create a callback that saves the model's weights during training
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,  # Save only the best model based on validation loss
    monitor="val_loss",
    mode="min",
    verbose=1
)

# Load the pre-trained ResNet50 model without the top classification layer
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Get number of classes
num_classes = len(dataset_labels[1])

# Add custom layers for classification
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_images, train_labels,
    batch_size=32,
    epochs=20,
    validation_data=(val_images, val_labels),
    callbacks=[checkpoint_callback]  # Add the callback here
)

Epoch 1/20
2/2 [==============================] - ETA: 0s - loss: 3.0906 - accuracy: 0.3226 
Epoch 1: val_loss improved from inf to 0.98544, saving model to C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/checkpoints\resnet50_face_recognition.h5
2/2 [==============================] - 16s 5s/step - loss: 3.0906 - accuracy: 0.3226 - val_loss: 0.9854 - val_accuracy: 0.6250
Epoch 2/20
2/2 [==============================] - ETA: 0s - loss: 0.2918 - accuracy: 0.9194
Epoch 2: val_loss improved from 0.98544 to 0.09335, saving model to C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/checkpoints\resnet50_face_recognition.h5
2/2 [==============================] - 8s 4s/step - loss: 0.2918 - accuracy: 0.9194 - val_loss: 0.0933 - val_accuracy: 0.9375
Epoch 3/20
2/2 [==============================] - ETA: 0s - loss: 0.0942 - accuracy: 0.9355
Epoch 3: val_loss im

### Evaluate Finetuned Model

In [65]:
def predict_test_images(test_dir, model):
    # Get all image paths in test directory
    image_paths = [os.path.join(test_dir, f) for f in os.listdir(test_dir) 
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    predictions = []
    
    for img_path in image_paths:
        # Load and preprocess image
        image = cv2.imread(img_path)
        faces, bounding_boxes = align_faces(image)  # use dlib for face detection and alignment

        if not faces:
            print(f"No face found, hence skipping.")
            return

        if len(faces) > 1:
            print(f"Multiple faces found, hence skipping.")
            return

        resized_face = cv2.resize(faces[0], (224, 224))
        resized_face = np.expand_dims(resized_face, axis=0)
        
        # Make prediction
        pred = model.predict(resized_face)  # Add batch dimension
        pred_class = np.argmax(pred, axis=1)[0]  # Get the predicted class index
        confidence = np.max(pred)  # Get the confidence score
        
        predictions.append({
            'image_path': img_path,
            'predicted_class': pred_class,
            'confidence': confidence
        })
        
        print(f"Image: {os.path.basename(img_path)}")
        print(f"Predicted class: {pred_class}")
        print(f"Confidence: {confidence:.4f}")
        print("---")
    
    return predictions

In [66]:
checkpoint_path = BASE + "checkpoints/resnet50_face_recognition.h5"
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

if os.path.exists(checkpoint_path):
	print(f"Loading checkpoint from `{checkpoint_path}`...")
	# Add custom layers again for classification
	x = Flatten()(base_model.output)
	x = Dense(128, activation='relu')(x)
	x = Dropout(0.5)(x)
	predictions = Dense(num_classes, activation='softmax')(x)

	# Define the new model (with the custom layers)
	model = Model(inputs=base_model.input, outputs=predictions)

	# Load the weights from the checkpoint file
	model.load_weights(checkpoint_path)
else:
	print(f"Checkpoint `{checkpoint_path}` not found.")

Loading checkpoint from `C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/checkpoints/resnet50_face_recognition.h5`...


In [67]:
test_dir = os.path.join(BASE, "test/")

predictions = predict_test_images(test_dir, model)


1/1 [==============================] - 0s 484ms/step
Image: 20250320_141825.jpg
Predicted class: 5
Confidence: 0.9994
---


In [68]:
print(label_map)

{'aaryan': 0, 'ethan': 1, 'eunice': 2, 'jinwei': 3, 'jonathan': 4, 'junyong': 5}
